In [94]:
from prophet import Prophet
from prophet.utilities import regressor_coefficients
from prophet.plot import add_changepoints_to_plot
from neuralprophet import NeuralProphet
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, root_mean_squared_error

In [26]:
day_data = pd.read_csv('../day_data.csv')
day_data = day_data.rename(columns={'date': 'ds'})
day_data['ds'] = pd.to_datetime(day_data['ds'], format="%Y-%m-%d")
day_data = day_data[day_data['ds'] < '2024-02-01']

holidays = day_data[['ds', 'holiday']]
holidays['lower_window'] = -7
holidays['upper_window'] = 7
holidays['ds'] = pd.to_datetime(holidays['ds'], format="%Y-%m-%d")
calendar = holidays
calendar = calendar[~calendar['holiday'].isnull()]

start_date = day_data['ds'].min()
end_date = '2024-01-31'
all_dates = pd.DataFrame(
    data=pd.date_range(start_date, end_date, freq='d'),
    columns=['ds']
)
all_dates['ds'] = pd.to_datetime(all_dates['ds'], format="%Y-%m-%d")

day_data = day_data.drop(columns=['holiday'])
regressor_columns = ['moon', 'arrest', 'wd']
output_data = pd.merge(
    all_dates[all_dates['ds'] >= day_data['ds'].min()], 
    day_data, 
    how='left', 
    on=['ds'])
output_data.sort_values(
    by=['ds'], 
    ascending=True, 
    inplace=True, 
    ignore_index=True)

input_data = day_data.rename(columns={'num_crimes': 'y'})
enc = OneHotEncoder(drop='first')
wd_enc = enc.fit_transform(input_data[['wd']])
wd_enc = wd_enc.toarray()
wds = enc.get_feature_names_out(['wd'])
wd_enc = pd.DataFrame(wd_enc, columns=wds)
input_data = pd.concat([input_data, wd_enc], axis=1)
input_data = input_data.drop(columns=['wd'])
input_data = input_data[input_data['ds'] < '2024-01-01']

output_data = output_data.rename(columns={'num_crimes': 'y'})
wd_enc = enc.fit_transform(output_data[['wd']])
wds = enc.get_feature_names_out(['wd'])
wd_enc = wd_enc.toarray()
wd_enc = pd.DataFrame(wd_enc, columns=wds)
output_data = pd.concat([output_data, wd_enc], axis=1)
output_data = output_data.drop(columns=['wd'])

proph = Prophet(
    holidays=calendar,
    yearly_seasonality=True,
    seasonality_mode='multiplicative',
    holidays_prior_scale=0.05,
    changepoint_prior_scale=0.05,
    uncertainty_samples=50)

regressors = ['moon', 'arrest', 'wPC1', 'wPC2', 'wPC3']
for r in regressors:
    proph.add_regressor(r)
input_data = input_data[['ds', 'y', 'moon', 'arrest', 'wPC1', 'wPC2', 'wPC3']]
proph.fit(input_data)
forecast = proph.predict(output_data)

# pred accuracy
preds = forecast[forecast['ds'] > '2023-12-31']['yhat']
y_real = output_data[output_data['ds'] > '2023-12-31']['y']
print(f' mse: {mean_squared_error(y_real, preds)}')
print(f' rmse: {root_mean_squared_error(y_real, preds)}')

# plots
for_plot = proph.plot(forecast)
a = add_changepoints_to_plot(for_plot.gca(), proph, forecast)
comps = proph.plot_components(forecast)

C:\Users\benke\AppData\Local\Temp\ipykernel_33400\827163187.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holidays['lower_window'] = -7
C:\Users\benke\AppData\Local\Temp\ipykernel_33400\827163187.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holidays['upper_window'] = 7
C:\Users\benke\AppData\Local\Temp\ipykernel_33400\827163187.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [95]:
regressor_coefficients(proph)

,regressor,regressor_mode,center,coef_lower,coef,coef_upper
0,moon,multiplicative,0.000000,-0.039901,-0.039901,-0.039901
1,arrest,multiplicative,0.372632,0.289941,0.289941,0.289941
2,wPC1,multiplicative,-0.037640,-0.032090,-0.032090,-0.032090
3,wPC2,multiplicative,-0.005754,0.003034,0.003034,0.003034
4,wPC3,multiplicative,-0.028530,0.027305,0.027305,0.027305


In [93]:
# Neural Prophet

moon_df = input_data[input_data['moon'] == 1][['ds']]
moon_df['event'] = 'full_moon'
calendar_np = calendar[['ds', 'holiday']]
calendar_np = calendar_np.rename(columns={'holiday': 'event'})

np = NeuralProphet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    seasonality_mode='multiplicative',
    n_lags=10,
    quantiles=[0.025, 0.975],
    learning_rate=0.0001,
    epochs=100
)
np.add_events("full_moon")
np.add_country_holidays("US", lower_window=-7, upper_window=7)
np.add_lagged_regressor('wPC1', n_lags=3)
np.add_lagged_regressor('wPC2', n_lags=3)
np.add_lagged_regressor('wPC3', n_lags=3)
np.add_lagged_regressor('arrest', n_lags=3)
input_all = np.create_df_with_events(input_data, moon_df)
input_all = input_all.drop(columns=['moon'])

np.fit(input_all)

output_data_np = output_data[['ds', 'y', 'arrest', 'wPC1', 'wPC2', 'wPC3']]
output_data_np
output_all = np.create_df_with_events(output_data_np, moon_df)

forecast = np.predict(output_all)

# pred accuracy
preds = forecast[forecast['ds'] > '2023-12-31']['yhat1']
y_real = output_data[output_data['ds'] > '2023-12-31']['y']
print(f' mse: {mean_squared_error(y_real, preds)}')
print(f' rmse: {root_mean_squared_error(y_real, preds)}')

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
c:\Users\benke\anaconda3\envs\durham\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.98% of the data.
INFO - (NP.df_utils._infer_frequency) - Dataframe freq automatically defined as D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
c:\Users\benke\anaconda3\envs\durham\Lib\site-packages\neuralprophet\time_d

Epoch 70: 100%|██████████| 70/70 [00:02<00:00, 30.47it/s, loss=0.0206, v_num=3, MAE=6.720, RMSE=8.680, Loss=0.0214, RegLoss=0.000]   


INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
c:\Users\benke\anaconda3\envs\durham\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.981% of the data.
c:\Users\benke\anaconda3\envs\durham\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)
c:\Users\benke\anaconda3\envs\durham\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_

Predicting DataLoader 0: 100%|██████████| 6/6 [00:00<00:00, 11.45it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column



 mse: 37.057459549559304
 rmse: 6.087483843884869
